In [1]:
print('hello world')

hello world


In [4]:
with open('../data/bible.txt') as f:
    text = f.read()

In [20]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#&()*,-./0123456789:;?@PR[]_abcdehilmnoprtuvy «»АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё—…
117


In [32]:
ctoi = {c : i for i, c in enumerate(chars)}
itoc = {i : c for i, c in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i.item()] if type(i) is torch.Tensor else itoc[i] for i in l])

encoded_hi = encode('привет')
print(encoded_hi)
print(decode(encoded_hi))

[97, 98, 90, 84, 87, 100]
привет


In [34]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(decode(data[100:110]))

torch.Size([4147059])
лю.

2 Зем


In [35]:
# train test split
train_size_p = 0.9
n = int(len(data) * train_size_p)
train_data = data[:n]
val_data = data[n:]

In [36]:
block_size = 8
train_data[:block_size+1]

tensor([ 53,  87, 100, 103,  90,  91,   1,  89,  82])

In [38]:
torch.manual_seed(42)

batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        print(f'when context is {xb[b, :t+1]} the answer is {yb[b,t]}')


when context is tensor([1]) the answer is 96
when context is tensor([ 1, 96]) the answer is 83
when context is tensor([ 1, 96, 83]) the answer is 98
when context is tensor([ 1, 96, 83, 98]) the answer is 82
when context is tensor([ 1, 96, 83, 98, 82]) the answer is 100
when context is tensor([  1,  96,  83,  98,  82, 100]) the answer is 90
when context is tensor([  1,  96,  83,  98,  82, 100,  90]) the answer is 93
when context is tensor([  1,  96,  83,  98,  82, 100,  90,  93]) the answer is 90
when context is tensor([1]) the answer is 86
when context is tensor([ 1, 86]) the answer is 87
when context is tensor([ 1, 86, 87]) the answer is 95
when context is tensor([ 1, 86, 87, 95]) the answer is 110
when context is tensor([  1,  86,  87,  95, 110]) the answer is 1
when context is tensor([  1,  86,  87,  95, 110,   1]) the answer is 97
when context is tensor([  1,  86,  87,  95, 110,   1,  97]) the answer is 96
when context is tensor([  1,  86,  87,  95, 110,   1,  97,  96]) the answer 

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):
        logits = self.embeddings(idx) # batch x block x vocab_size

        if target is None:
            loss = None
        else:
            logits = logits.view(batch_size * block_size, vocab_size)
            target = target.view(batch_size * block_size)
            loss = F.cross_entropy(logits, target)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # batch x block x vocab_size
            logits = logits[:, -1, :] # batch x vocab_size
            probs = F.softmax(logits, dim=-1) # batch x vocab_size
            idx_next = torch.multinomial(probs, num_samples=1) # batch x 1
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramModel()
logits, loss = m(xb, yb)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), 100)[0].tolist()))

tensor(4.9712, grad_fn=<NllLossBackward0>)

жТиЭИyБRэoВПдй Р!y1Тё егг]dус 2mRёХКж)Ь44v)ТtР6Эc/_Хrн_ърЦйИyМvt/5э5*Тri«4Э]9ТМооЕ«НУф6МлХЗчСКc&АхГм


In [46]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [50]:
batch_size = 32
N_STEPS = 10000

for step in range(N_STEPS):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4091498851776123


In [52]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), 500)[0].tolist()))


461 Ж8 Говсь х и по я стоги вь, и и тьтсим, оеважил иЦю еда бодобоей, сомли ст изв м, казахеливаспрьвих, Ах сто нала Оноршерита и м? Модре И вёлоиренидми си в ни в ви нане, чей пае.
15 Иешими ух, кои ть беры с пожуенараждаролоти бынавстелобого трене. е вомелешла и го пт у» пршёси и сля:

166ще, илсе см

18 И г какудегота, у одагил веду.
2 Свони зылнинелови Си гудт ча вотсази мни итя Егушомётвкотен к приевямо ет в огляза м.
3 Иай;
36за уткодеша и сей? оя

4 Ие онаме, Его килою, ко ихлего еля.

3 


In [54]:
batch_size = 4
block_size = 8
vocab_size = 2
x = torch.randn(batch_size, block_size, vocab_size)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.randn(batch_size, block_size, vocab_size)
for b in range(batch_size):
    for t in range(block_size):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [60]:
a = torch.tril(torch.ones(4, 4))
b = torch.randint(0, 10, (4, 3)).float()
c = a @ b
print(b)
print(c)

tensor([[7., 5., 2.],
        [5., 7., 4.],
        [0., 7., 8.],
        [9., 8., 8.]])
tensor([[ 7.,  5.,  2.],
        [12., 12.,  6.],
        [12., 19., 14.],
        [21., 27., 22.]])
